In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(0)

plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 12
plt.figsize = (10, 20)

# Helper Function
def display_image(image, title='Press Escape to Quit'):
    cv.imshow(title, image)
    # Press escape to close the image
    if cv.waitKey(0) & 0xff == 27:
        cv.destroyAllWindows()

1. On the following image, apply:
    - Mean (3x3)
    - Median (3x3)
    - Gaussian (5x5) filter $\sigma = 2$
    - Sharpening filter (as defined in my slide no. 19 of Lecture 04)

    Save the image after each type of filter along with original image (All 5 images on the one or two page) in a pdf file. Also provide details of the filters that you used in your answer.
    ![](input/input.png)

In [16]:
image = cv.imread("input/input.png", cv.IMREAD_GRAYSCALE)

In [17]:
def median_filter(image, kernel_size = 3):
    height, width = image.shape
    modified_image = np.zeros_like(image)
    
    for y in range(height):
        for x in range(width):
            neighborhood = image[y:y+kernel_size, x:x+kernel_size]
            modified_image[y, x] = np.median(neighborhood)
            
    return modified_image

def apply_kernel(image, kernel):
  height, width = image.shape
  kernel_height, kernel_width = kernel.shape
  pad_h, pad_w = kernel_height // 2, kernel_width // 2
  
  # Pad the entire image at once using edge padding
  padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='edge')
  modified_image = np.zeros_like(image)
  
  for y in range(height):
    for x in range(width):
      neighborhood = padded_image[y:y+kernel_height, x:x+kernel_width]
      modified_image[y, x] = np.sum(neighborhood * kernel)
      
  return modified_image

def create_gaussian_blur_kernel(size, sigma):
    half_block = size // 2
    one_d_kernel = np.exp(-np.arange(-half_block, half_block+1)**2 / (2 * sigma**2))
    two_d_kernel = np.outer(one_d_kernel, one_d_kernel)
    normalized_gaussian_kernel = two_d_kernel / np.sum(two_d_kernel)
    return normalized_gaussian_kernel
gaussian_kernel = create_gaussian_blur_kernel(5, 2)

In [18]:
mean_kernel = np.ones((3, 3)) / 9
sharpening_kernel = np.diag([0, 2, 0]) - np.ones((3, 3)) / 9

In [19]:
mean_image = apply_kernel(image, mean_kernel)
median_image = median_filter(image, 3)
gaussian_image = apply_kernel(image, gaussian_kernel)
sharpened_image = apply_kernel(image, sharpening_kernel)
cv.imwrite("output/singular/mean.png", mean_image)
cv.imwrite("output/singular/median.png", median_image)
cv.imwrite("output/singular/gaussian.png", gaussian_image)
cv.imwrite("output/singular/sharpened.png", sharpened_image)
print("Done")

Done


In [20]:
# Incrementally adding the filters
cv.imwrite("output/incremental/0.identity.png", image)
incremental_image = apply_kernel(image, mean_kernel)
cv.imwrite("output/incremental/1.mean.png", incremental_image)
incremental_image = median_filter(incremental_image, 3)
cv.imwrite("output/incremental/2.median_mean.png", incremental_image)
incremental_image = apply_kernel(incremental_image, gaussian_kernel)
cv.imwrite("output/incremental/3.gaussian_median_mean.png", incremental_image)
incremental_image = apply_kernel(incremental_image, sharpening_kernel)
cv.imwrite("output/incremental/4.sharpened_gaussian_median_mean.png", incremental_image)
print("Done")

Done
